In [15]:
%reload_ext nb_black
import numpy as np
import pandas as pd
import sklearn
import spacy
import re
import nltk
from nltk.corpus import gutenberg
import gensim
import warnings

warnings.filterwarnings("ignore")

nltk.download("gutenberg")
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\jlim7\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[x] Couldn't link model to 'en'
Creating a symlink in spacy/data failed. Make sure you have the required
permissions and try re-running the command as admin, or use a virtualenv. You
can still import the model as a module and call its load() method, or create the
symlink manually.
C:\Users\jlim7\anaconda3\lib\site-packages\en_core_web_sm -->
C:\Users\jlim7\anaconda3\lib\site-packages\spacy\data\en
[!] Download successful but linking failed
Creating a shortcut link for 'en' didn't work (maybe you don't have admin
permissions?), but you can still load the model via its full package name: nlp =
spacy.load('en_core_web_sm')


You do not have sufficient privilege to perform this operation.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# utility function for standard text cleaning
def text_cleaner(text):
    # visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r"--", " ", text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = " ".join(text.split())
    return text

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
# load and clean the data
persuasion = gutenberg.raw("austen-persuasion.txt")
alice = gutenberg.raw("carroll-alice.txt")

# the chapter indicator is idiosyncratic
persuasion = re.sub(r"Chapter \d+", "", persuasion)
alice = re.sub(r"CHAPTER .*", "", alice)

alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
# parse the cleaned novels. This can take a bit.
nlp = spacy.load("en_core_web_sm")
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
# group into sentences
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# combine the sentences from the two novels into one data frame
sentences = pd.DataFrame(alice_sents + persuasion_sents, columns=["text", "author"])
sentences.head()

,text,author
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
# get rid off stop words and punctuation
# and lemmatize the tokens
for i, sentence in enumerate(sentences["text"]):
    sentences.loc[i, "text"] = [
        token.lemma_ for token in sentence if not token.is_punct and not token.is_stop
    ]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
# train word2vec on the the sentences
model1 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=4,
    sg=0,
    sample=1e-3,
    size=100,
    hs=1,
)

model2 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=6,
    sg=0,
    sample=1e-3,
    size=100,
    hs=1,
)

model3 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=8,
    sg=0,
    sample=1e-3,
    size=100,
    hs=1,
)

model4 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=4,
    sg=0,
    sample=1e-3,
    size=300,
    hs=1,
)

model5 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=6,
    sg=0,
    sample=1e-3,
    size=300,
    hs=1,
)

model6 = gensim.models.Word2Vec(
    sentences["text"],
    workers=4,
    min_count=1,
    window=8,
    sg=0,
    sample=1e-3,
    size=300,
    hs=1,
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
word2vec_arr1 = np.zeros((sentences.shape[0], 100))
word2vec_arr2 = np.zeros((sentences.shape[0], 100))
word2vec_arr3 = np.zeros((sentences.shape[0], 100))
word2vec_arr4 = np.zeros((sentences.shape[0], 300))
word2vec_arr5 = np.zeros((sentences.shape[0], 300))
word2vec_arr6 = np.zeros((sentences.shape[0], 300))

for i, sentence in enumerate(sentences["text"]):
    word2vec_arr1[i, :] = np.mean([model1[lemma] for lemma in sentence], axis=0)
    word2vec_arr2[i, :] = np.mean([model2[lemma] for lemma in sentence], axis=0)
    word2vec_arr3[i, :] = np.mean([model3[lemma] for lemma in sentence], axis=0)
    word2vec_arr4[i, :] = np.mean([model4[lemma] for lemma in sentence], axis=0)
    word2vec_arr5[i, :] = np.mean([model5[lemma] for lemma in sentence], axis=0)
    word2vec_arr6[i, :] = np.mean([model6[lemma] for lemma in sentence], axis=0)

word2vec_arr1 = pd.DataFrame(word2vec_arr1)
word2vec_arr2 = pd.DataFrame(word2vec_arr2)
word2vec_arr3 = pd.DataFrame(word2vec_arr3)
word2vec_arr4 = pd.DataFrame(word2vec_arr4)
word2vec_arr5 = pd.DataFrame(word2vec_arr5)
word2vec_arr6 = pd.DataFrame(word2vec_arr6)

sentences1 = pd.concat([sentences[["author", "text"]], word2vec_arr1], axis=1)
sentences1.dropna(inplace=True)

sentences2 = pd.concat([sentences[["author", "text"]], word2vec_arr2], axis=1)
sentences2.dropna(inplace=True)

sentences3 = pd.concat([sentences[["author", "text"]], word2vec_arr3], axis=1)
sentences3.dropna(inplace=True)

sentences4 = pd.concat([sentences[["author", "text"]], word2vec_arr4], axis=1)
sentences4.dropna(inplace=True)

sentences5 = pd.concat([sentences[["author", "text"]], word2vec_arr5], axis=1)
sentences5.dropna(inplace=True)

sentences6 = pd.concat([sentences[["author", "text"]], word2vec_arr6], axis=1)
sentences6.dropna(inplace=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

Y1 = sentences1['author']
Y2 = sentences2['author']
Y3 = sentences3['author']
Y4 = sentences4['author']
Y5 = sentences5['author']
Y6 = sentences6['author']

X1 = np.array(sentences1.drop(['text','author'], 1))
X2 = np.array(sentences2.drop(['text','author'], 1))
X3 = np.array(sentences3.drop(['text','author'], 1))
X4 = np.array(sentences4.drop(['text','author'], 1))
X5 = np.array(sentences5.drop(['text','author'], 1))
X6 = np.array(sentences6.drop(['text','author'], 1))

# We split the dataset into train and test sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.4, random_state=123)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size=0.4, random_state=123)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, Y3, test_size=0.4, random_state=123)
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, Y4, test_size=0.4, random_state=123)
X_train5, X_test5, y_train5, y_test5 = train_test_split(X5, Y5, test_size=0.4, random_state=123)
X_train6, X_test6, y_train6, y_test6 = train_test_split(X6, Y6, test_size=0.4, random_state=123)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

print("-----------------------Word2vec Model 1------------------------------")
lr.fit(X_train1, y_train1)
rfc.fit(X_train1, y_train1)
gbc.fit(X_train1, y_train1)
print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train1, y_train1))
print('\nTest set score:', lr.score(X_test1, y_test1))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train1, y_train1))
print('\nTest set score:', rfc.score(X_test1, y_test1))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train1, y_train1))
print('\nTest set score:', gbc.score(X_test1, y_test1))

print("-----------------------Word2vec Model 2------------------------------")
lr.fit(X_train2, y_train2)
rfc.fit(X_train2, y_train2)
gbc.fit(X_train2, y_train2)
print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train2, y_train2))
print('\nTest set score:', lr.score(X_test2, y_test2))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train2, y_train2))
print('\nTest set score:', rfc.score(X_test2, y_test2))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train2, y_train2))
print('\nTest set score:', gbc.score(X_test2, y_test2))

print("-----------------------Word2vec Model 3------------------------------")
lr.fit(X_train3, y_train3)
rfc.fit(X_train3, y_train3)
gbc.fit(X_train3, y_train3)
print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train3, y_train3))
print('\nTest set score:', lr.score(X_test3, y_test3))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train3, y_train3))
print('\nTest set score:', rfc.score(X_test3, y_test3))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train3, y_train3))
print('\nTest set score:', gbc.score(X_test3, y_test3))

print("-----------------------Word2vec Model 4------------------------------")
lr.fit(X_train4, y_train4)
rfc.fit(X_train4, y_train4)
gbc.fit(X_train4, y_train4)
print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train4, y_train4))
print('\nTest set score:', lr.score(X_test4, y_test4))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train4, y_train4))
print('\nTest set score:', rfc.score(X_test4, y_test4))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train4, y_train4))
print('\nTest set score:', gbc.score(X_test4, y_test4))

print("-----------------------Word2vec Model 5------------------------------")
lr.fit(X_train5, y_train5)
rfc.fit(X_train5, y_train5)
gbc.fit(X_train5, y_train5)
print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train5, y_train5))
print('\nTest set score:', lr.score(X_test5, y_test5))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train5, y_train5))
print('\nTest set score:', rfc.score(X_test5, y_test5))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train5, y_train5))
print('\nTest set score:', gbc.score(X_test5, y_test5))

print("-----------------------Word2vec Model 6------------------------------")
lr.fit(X_train6, y_train6)
rfc.fit(X_train6, y_train6)
gbc.fit(X_train6, y_train6)
print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train6, y_train6))
print('\nTest set score:', lr.score(X_test6, y_test6))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train6, y_train6))
print('\nTest set score:', rfc.score(X_test6, y_test6))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train6, y_train6))
print('\nTest set score:', gbc.score(X_test6, y_test6))


-----------------------Word2vec Model 1------------------------------
----------------------Logistic Regression Scores----------------------
Training set score: 0.8006819590824551

Test set score: 0.8089260808926081
----------------------Random Forest Scores----------------------
Training set score: 0.9950402975821451

Test set score: 0.8410041841004184
----------------------Gradient Boosting Scores----------------------
Training set score: 0.8946063236205828

Test set score: 0.8317061831706183
-----------------------Word2vec Model 2------------------------------
----------------------Logistic Regression Scores----------------------
Training set score: 0.7978921264724117

Test set score: 0.8047419804741981
----------------------Random Forest Scores----------------------
Training set score: 0.9950402975821451

Test set score: 0.8354253835425384
----------------------Gradient Boosting Scores----------------------
Training set score: 0.8992560446373218

Test set score: 0.8428637842863784


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>